# Annual Water Use 

In [1]:

import os, sys
import pandas as pd

from iwateruse.featurize import MultiOneHotEncoder
import matplotlib.pyplot as plt
from xgboost import plot_importance
import xgboost as xgb
import category_encoders as ce 
from lightgbm import LGBMRegressor

#
import numpy as np

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

C:\Miniconda3\envs\py39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Miniconda3\envs\py39\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
%matplotlib widget
%matplotlib inline
%matplotlib ipympl
import warnings

warnings.filterwarnings('ignore')
xgb.set_config(verbosity=0)

In [4]:
if 0: #only when pop is not in the database
    
    dataset = pd.read_csv(r"C:\work\water_use\ml_experiments\annual_v_0_0\clean_train_db.csv")
    master_pop = pd.read_csv(r"master_population.csv")
    master_pop.rename(columns = {'WSA_AGIDF':'sys_id'}, inplace = True)
    master_pop = master_pop[['sys_id', 'pop', 'Year']]
    master_pop = master_pop.groupby(by=['sys_id', 'Year']).mean()
    master_pop.reset_index(inplace=True)


    #add population
    dataset = dataset.merge(master_pop, on=['sys_id', 'Year'], how = 'left')
    mask = dataset['pop'].isna()
    dataset.loc[mask, 'pop'] = dataset[mask]['population']
    dataset.to_csv(r"C:\work\water_use\ml_experiments\annual_v_0_0\clean_train_db.csv")

In [5]:
dataset = pd.read_csv(r"C:\work\water_use\ml_experiments\annual_v_0_0\clean_train_db.csv")
# pop_info = pd.read_csv(r"pop_info.csv")
#pc_50_swud = pd.read_csv(r"C:\work\water_use\mldataset\ml\training\misc_features\spatial_features\pc_50_pop_swud15.csv")
pc_50_swud = pd.read_csv(r"spatial_pc_statistics_apply_max_min_pc.csv")
# pc_50_plc = pd.read_csv(r"C:\work\water_use\mldataset\ml\training\misc_features\spatial_features\pc_50_plc_pop_interpolated.csv")
# pc95_5 = pd.read_csv(r"C:\work\water_use\mldataset\ml\training\misc_features\spatial_features\pc_50_swud_pop_p95_5.csv")

In [6]:
dataset

,population,pop_density,gini,n_employed,n_occupation,n_occ_management,n_occ_service,n_occ_sales_office,n_occ_farm_fish_forest,n_occ_const_maint_repair,...,WSA_SQKM,KG_climate_zone,prc_n_lt_ninth_gr,prc_n_ninth_to_twelth_gr,prc_n_hs_grad,prc_n_some_college,prc_n_associates,prc_n_bachelors,prc_n_masters_phd,pop
0,8798.870753,3.098065e+06,0.403000,7055.896367,4790.929562,1950.971035,553.992129,1362.979793,475.993095,446.993509,...,10.59,25,2.301500,6.115406,34.505998,13.397996,8.531972,22.127237,13.019891,9288.0
1,264.918028,5.093688e+05,0.335600,131.481262,60.261158,29.621071,-9.025729,15.813636,11.451154,-2.758788,...,2.12,25,83.345627,80.464142,104.515114,93.069328,69.345421,94.484520,72.142677,467.0
2,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,437.07,25,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0
3,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,437.07,25,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0
4,60465.995810,3.214985e+06,0.367000,49116.996600,31471.997820,9993.999309,5768.999593,7876.999458,3268.999774,4562.999689,...,69.44,25,4.980173,7.487754,39.010963,19.241894,8.019594,13.711220,7.548402,60491.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78644,1051.115835,8.023602e+05,0.358300,769.132864,394.776160,59.313660,70.009565,111.820834,59.313660,94.318442,...,5.50,14,2.140673,11.773700,40.825688,27.981651,6.880734,6.574924,3.822630,718.0
78645,957.769748,7.311052e+05,0.407000,785.662900,375.329059,72.926631,67.092500,115.710254,41.811268,77.788406,...,5.50,14,2.954210,13.146233,39.143279,24.815362,8.271787,7.976366,3.692762,725.0
78646,1082.231198,8.261118e+05,0.389900,873.174857,461.868660,91.401377,71.954275,157.521522,58.341304,82.650181,...,5.50,14,5.503356,14.496644,32.348993,23.758389,9.127517,8.053691,6.711409,731.0
78647,1274.757502,9.730751e+05,0.337500,1007.359857,641.754349,83.622536,75.843696,208.083986,68.064855,206.139276,...,5.50,14,4.785276,11.288344,38.895706,17.300614,8.711656,15.828221,3.190184,743.0


In [7]:
df_ = pc_50_swud[['sys_id', 'pop_tmean' ]]
dataset = dataset.merge(df_, right_on=['sys_id'], left_on=['sys_id'] , how = 'left')
del(df_)

#df_ = pc95_5[['sys_id', 'p95', 'p5' ]]
#dataset = dataset.merge(df_, right_on=['sys_id'], left_on=['sys_id'] , how = 'left')
#del(df_)

In [10]:
# make dataset
master_pop = pd.read_csv(r"master_population.csv")
annual_wu = pd.read_csv(r"annual_wu.csv")

In [11]:
dataset

,population,pop_density,gini,n_employed,n_occupation,n_occ_management,n_occ_service,n_occ_sales_office,n_occ_farm_fish_forest,n_occ_const_maint_repair,...,KG_climate_zone,prc_n_lt_ninth_gr,prc_n_ninth_to_twelth_gr,prc_n_hs_grad,prc_n_some_college,prc_n_associates,prc_n_bachelors,prc_n_masters_phd,pop,pop_tmean
0,8798.870753,3.098065e+06,0.403000,7055.896367,4790.929562,1950.971035,553.992129,1362.979793,475.993095,446.993509,...,25,2.301500,6.115406,34.505998,13.397996,8.531972,22.127237,13.019891,9288.0,71.276372
1,264.918028,5.093688e+05,0.335600,131.481262,60.261158,29.621071,-9.025729,15.813636,11.451154,-2.758788,...,25,83.345627,80.464142,104.515114,93.069328,69.345421,94.484520,72.142677,467.0,77.471245
2,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,25,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832
3,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,25,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832
4,60465.995810,3.214985e+06,0.367000,49116.996600,31471.997820,9993.999309,5768.999593,7876.999458,3268.999774,4562.999689,...,25,4.980173,7.487754,39.010963,19.241894,8.019594,13.711220,7.548402,60491.0,70.663169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78644,1051.115835,8.023602e+05,0.358300,769.132864,394.776160,59.313660,70.009565,111.820834,59.313660,94.318442,...,14,2.140673,11.773700,40.825688,27.981651,6.880734,6.574924,3.822630,718.0,153.227055
78645,957.769748,7.311052e+05,0.407000,785.662900,375.329059,72.926631,67.092500,115.710254,41.811268,77.788406,...,14,2.954210,13.146233,39.143279,24.815362,8.271787,7.976366,3.692762,725.0,153.227055
78646,1082.231198,8.261118e+05,0.389900,873.174857,461.868660,91.401377,71.954275,157.521522,58.341304,82.650181,...,14,5.503356,14.496644,32.348993,23.758389,9.127517,8.053691,6.711409,731.0,153.227055
78647,1274.757502,9.730751e+05,0.337500,1007.359857,641.754349,83.622536,75.843696,208.083986,68.064855,206.139276,...,14,4.785276,11.288344,38.895706,17.300614,8.711656,15.828221,3.190184,743.0,153.227055


In [12]:

#master_pop

In [13]:
annual_wu['wu_rate_mean'] = annual_wu[['annual_wu_G_swuds', 'annual_wu_G_nonswuds']].mean(axis = 1)
annual_wu['wu_rate_mean']= annual_wu['wu_rate_mean']/annual_wu['days_in_year']
avg_wu = annual_wu[['WSA_AGIDF', 'YEAR', 'wu_rate_mean' ]].copy()
avg_wu.rename(columns = {'WSA_AGIDF':'sys_id', 'YEAR':'Year', 'wu_rate_mean':'wu_rate'}, inplace = True)

In [14]:
master_pop.rename(columns = {'WSA_AGIDF':'sys_id'}, inplace = True)
master_pop = master_pop[['sys_id', 'pop', 'Year']]
master_pop = master_pop.groupby(by=['sys_id', 'Year']).mean()
master_pop.reset_index(inplace=True)


#add population
if 0:
    dataset = dataset.merge(master_pop, on=['sys_id', 'Year'], how = 'left')
    mask = dataset['pop'].isna()
    dataset.loc[mask, 'pop'] = dataset[mask]['population']
#add water use
del(dataset['wu_rate'])
dataset = dataset.merge(avg_wu, on=['sys_id', 'Year'], how = 'left')

In [15]:
dataset

,population,pop_density,gini,n_employed,n_occupation,n_occ_management,n_occ_service,n_occ_sales_office,n_occ_farm_fish_forest,n_occ_const_maint_repair,...,prc_n_lt_ninth_gr,prc_n_ninth_to_twelth_gr,prc_n_hs_grad,prc_n_some_college,prc_n_associates,prc_n_bachelors,prc_n_masters_phd,pop,pop_tmean,wu_rate
0,8798.870753,3.098065e+06,0.403000,7055.896367,4790.929562,1950.971035,553.992129,1362.979793,475.993095,446.993509,...,2.301500,6.115406,34.505998,13.397996,8.531972,22.127237,13.019891,9288.0,71.276372,9.268000e+05
1,264.918028,5.093688e+05,0.335600,131.481262,60.261158,29.621071,-9.025729,15.813636,11.451154,-2.758788,...,83.345627,80.464142,104.515114,93.069328,69.345421,94.484520,72.142677,467.0,77.471245,7.100000e+03
2,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
3,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
4,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85206,1051.115835,8.023602e+05,0.358300,769.132864,394.776160,59.313660,70.009565,111.820834,59.313660,94.318442,...,2.140673,11.773700,40.825688,27.981651,6.880734,6.574924,3.822630,718.0,153.227055,1.917316e+05
85207,957.769748,7.311052e+05,0.407000,785.662900,375.329059,72.926631,67.092500,115.710254,41.811268,77.788406,...,2.954210,13.146233,39.143279,24.815362,8.271787,7.976366,3.692762,725.0,153.227055,2.137436e+05
85208,1082.231198,8.261118e+05,0.389900,873.174857,461.868660,91.401377,71.954275,157.521522,58.341304,82.650181,...,5.503356,14.496644,32.348993,23.758389,9.127517,8.053691,6.711409,731.0,153.227055,1.659505e+05
85209,1274.757502,9.730751e+05,0.337500,1007.359857,641.754349,83.622536,75.843696,208.083986,68.064855,206.139276,...,4.785276,11.288344,38.895706,17.300614,8.711656,15.828221,3.190184,743.0,153.227055,NaN


In [16]:
np.sum(dataset['pop'].isna())

0

In [17]:
dataset = dataset[dataset['wu_rate']>0]
np.sum(dataset['wu_rate'].isna())

0

In [18]:
dataset

,population,pop_density,gini,n_employed,n_occupation,n_occ_management,n_occ_service,n_occ_sales_office,n_occ_farm_fish_forest,n_occ_const_maint_repair,...,prc_n_lt_ninth_gr,prc_n_ninth_to_twelth_gr,prc_n_hs_grad,prc_n_some_college,prc_n_associates,prc_n_bachelors,prc_n_masters_phd,pop,pop_tmean,wu_rate
0,8798.870753,3.098065e+06,0.403000,7055.896367,4790.929562,1950.971035,553.992129,1362.979793,475.993095,446.993509,...,2.301500,6.115406,34.505998,13.397996,8.531972,22.127237,13.019891,9288.0,71.276372,9.268000e+05
1,264.918028,5.093688e+05,0.335600,131.481262,60.261158,29.621071,-9.025729,15.813636,11.451154,-2.758788,...,83.345627,80.464142,104.515114,93.069328,69.345421,94.484520,72.142677,467.0,77.471245,7.100000e+03
2,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
3,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
4,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85205,898.456088,6.858287e+05,0.377500,640.781994,321.849530,49.590109,46.673044,95.290797,50.562464,79.733116,...,2.288732,13.028169,38.204225,27.816901,4.577465,9.330986,4.753521,712.0,153.227055,1.742274e+05
85206,1051.115835,8.023602e+05,0.358300,769.132864,394.776160,59.313660,70.009565,111.820834,59.313660,94.318442,...,2.140673,11.773700,40.825688,27.981651,6.880734,6.574924,3.822630,718.0,153.227055,1.917316e+05
85207,957.769748,7.311052e+05,0.407000,785.662900,375.329059,72.926631,67.092500,115.710254,41.811268,77.788406,...,2.954210,13.146233,39.143279,24.815362,8.271787,7.976366,3.692762,725.0,153.227055,2.137436e+05
85208,1082.231198,8.261118e+05,0.389900,873.174857,461.868660,91.401377,71.954275,157.521522,58.341304,82.650181,...,5.503356,14.496644,32.348993,23.758389,9.127517,8.053691,6.711409,731.0,153.227055,1.659505e+05


In [19]:
dataset_bck = dataset.copy()

In [20]:
# drop_before_preprocess = ['sys_id']
# categorical_features = ['HUC2', 'state_id',  'KG_climate_zone', 'county_id', 'Ecode_num' ]
# ohc1 = MultiOneHotEncoder(catfeatures = categorical_features )
# dataset = ohc1.transform(dataset)

In [21]:
dataset = dataset_bck.copy()

In [22]:
dataset

,population,pop_density,gini,n_employed,n_occupation,n_occ_management,n_occ_service,n_occ_sales_office,n_occ_farm_fish_forest,n_occ_const_maint_repair,...,prc_n_lt_ninth_gr,prc_n_ninth_to_twelth_gr,prc_n_hs_grad,prc_n_some_college,prc_n_associates,prc_n_bachelors,prc_n_masters_phd,pop,pop_tmean,wu_rate
0,8798.870753,3.098065e+06,0.403000,7055.896367,4790.929562,1950.971035,553.992129,1362.979793,475.993095,446.993509,...,2.301500,6.115406,34.505998,13.397996,8.531972,22.127237,13.019891,9288.0,71.276372,9.268000e+05
1,264.918028,5.093688e+05,0.335600,131.481262,60.261158,29.621071,-9.025729,15.813636,11.451154,-2.758788,...,83.345627,80.464142,104.515114,93.069328,69.345421,94.484520,72.142677,467.0,77.471245,7.100000e+03
2,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
3,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
4,391870.123700,3.531143e+06,0.475100,307138.207100,188791.782000,66959.796810,35032.642330,48278.679260,18074.291140,20446.372450,...,7.545115,8.337700,29.848587,16.870522,6.790972,17.623259,12.983845,871615.0,90.045832,4.704930e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85205,898.456088,6.858287e+05,0.377500,640.781994,321.849530,49.590109,46.673044,95.290797,50.562464,79.733116,...,2.288732,13.028169,38.204225,27.816901,4.577465,9.330986,4.753521,712.0,153.227055,1.742274e+05
85206,1051.115835,8.023602e+05,0.358300,769.132864,394.776160,59.313660,70.009565,111.820834,59.313660,94.318442,...,2.140673,11.773700,40.825688,27.981651,6.880734,6.574924,3.822630,718.0,153.227055,1.917316e+05
85207,957.769748,7.311052e+05,0.407000,785.662900,375.329059,72.926631,67.092500,115.710254,41.811268,77.788406,...,2.954210,13.146233,39.143279,24.815362,8.271787,7.976366,3.692762,725.0,153.227055,2.137436e+05
85208,1082.231198,8.261118e+05,0.389900,873.174857,461.868660,91.401377,71.954275,157.521522,58.341304,82.650181,...,5.503356,14.496644,32.348993,23.758389,9.127517,8.053691,6.711409,731.0,153.227055,1.659505e+05


In [23]:
categorical_features = ['HUC2', 'state_id']
# dataset = dataset.astype({"HUC2":'category', "state_id":'category',
#                       "KG_climate_zone":'category',
#                       "Ecode_num":'category',}) #"county_id":'category',

In [24]:
#ordered_features

In [25]:
extra_drop = []

In [26]:
#extra_drop = ordered_features[50:].to_list()
extra_drop = []
for i in ['KG_climate_zone', 'HUC2', 'state_id', 'Ecode_num', 'county_id', "KG_climate_zone"]:
    try:
        extra_drop.remove(i)
    except:
        print(" {} is not the list".format(i))

 KG_climate_zone is not the list
 HUC2 is not the list
 state_id is not the list
 Ecode_num is not the list
 county_id is not the list
 KG_climate_zone is not the list


In [ ]:

def log_cosh_quantile(alpha):
    def _log_cosh_quantile(y_true, y_pred):
        err = y_pred - y_true
        err = np.where(err < 0, alpha * err, (1 - alpha) * err)
        grad = np.tanh(err)
        hess = 1 / np.cosh(err)**2
        hess[hess<0.01] =0.01 
        #hess = np.ones_like(hess)
        
        return grad, hess
    return _log_cosh_quantile

In [27]:
columns_to_drop = [ 'population', 'wu_rate'] + extra_drop#'population',
df = dataset.copy()
df = df.dropna(axis = 0)


df
df['pc'] = df['wu_rate']/df['pop']
#df['pc'] = df['pc_median']
#del( df['pc_median'])
if 1:
    df = df[df['pop']>8000]
    mask = (df['pc']>=30) & (df['pc']<=400)
    df = df[mask]
    #df = df[df['LONG']<=-98]
    #df = df[df['pc']<df['p95']]
    #df = df[df['pc']>df['p5']]

#mask = ((df['pc_median']-df['pc'])/df['pc_median']) <0.5
#df = df[mask]

df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('pc')
X =df[features]
#from statsmodels.distributions.empirical_distribution import ECDF
# y = df['pc'].values
# ec = ECDF(y)
# y = ec(y)
y =df['pc']

# 
#plt.figure()
#v_freq = plt.hist(df['pc'].values, bins = 30)
#freq = np.interp(df['pc'].values, v_freq[1][1:], v_freq[0], left=None, right=None, period=None)
#X['freq'] = freq

tar_ec = ce.TargetEncoder(return_df=True, cols = ['county_id'] )
#X = tar_ec.fit_transform(X, y)

#Split data
if 1:
    del(X['sys_id'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 123)
    #Train_freq = X_train['freq']
    #Test_freq = X_test['freq']
    #del(X_test['freq'])
    #del(X_train['freq'])
else:
    if 0:
        east_sys = pd.DataFrame(df[df['LONG']>-98]['sys_id'].unique())
        west_sys = pd.DataFrame(df[df['LONG']<=-98]['sys_id'].unique())
        east_sys_train = east_sys.sample(frac = 0.7, random_state = 123)
        west_sys_train = west_sys.sample(frac = 0.7, random_state = 123)
        trainsys = east_sys_train[0].values.tolist() +  west_sys_train[0].values.tolist()
        east_testsys = east_sys.drop(east_sys_train.index)
        west_testsys = west_sys.drop(west_sys_train.index)
        testsys = east_testsys[0].values.tolist() +  west_testsys[0].values.tolist()
        
        X_train = df[df['sys_id'].isin(trainsys)]
        y_train = X_train['pc']
        del(X_train['pc'])
        del(X_train['sys_id'])

        X_test = df[df['sys_id'].isin(testsys)]
        y_test = X_test['pc']
        del(X_test['pc'])
        del(X_test['sys_id'])
    
    else:
        sys_names = pd.DataFrame(df['sys_id'].unique())
        trainsys = sys_names.sample(frac = 0.7, random_state = 123)
        testsys = sys_names.drop(trainsys.index)

        X_train = df[df['sys_id'].isin(trainsys[0])]
        y_train = X_train['pc']
        del(X_train['pc'])
        del(X_train['sys_id'])

        X_test = df[df['sys_id'].isin(testsys[0])]
        y_test = X_test['pc']
        del(X_test['pc'])
        del(X_test['sys_id'])

    



In [29]:
X_train

,pop_density,gini,n_employed,n_occupation,n_occ_management,n_occ_service,n_occ_sales_office,n_occ_farm_fish_forest,n_occ_const_maint_repair,n_occ_prod_trans_material,...,KG_climate_zone,prc_n_lt_ninth_gr,prc_n_ninth_to_twelth_gr,prc_n_hs_grad,prc_n_some_college,prc_n_associates,prc_n_bachelors,prc_n_masters_phd,pop,pop_tmean
50799,1621771.380,3.684128e-01,9329.472038,5501.940486,1191.053687,1347.004681,1212.593230,968.895419,782.393468,5501.940486,...,14,12.488156,13.119477,29.606810,19.951260,5.692442,13.364341,5.777514,13746.0,184.255598
47553,5836885.609,-1.452558e+06,545107.046600,380054.872300,183345.625600,43405.541250,84404.405810,59564.410620,60247.226800,911954.560100,...,14,6.183223,6.320588,20.614096,14.972118,5.431910,25.329708,21.148357,4846163.0,40.871268
54043,2382328.649,3.138839e-01,24150.644630,16290.557940,3868.856976,1849.474996,3502.156032,1204.128878,3987.098348,13439.473030,...,25,11.925154,14.900265,35.544012,16.958880,2.216817,10.647838,7.807034,30448.0,107.803743
42931,5611067.551,3.706800e-01,93082.527000,49677.062920,12313.506180,10369.533910,13113.440520,3817.262214,10063.320090,49677.062920,...,25,8.809227,15.155695,34.211371,17.081612,7.193114,11.048065,6.500916,124730.0,85.403447
48707,2032488.854,4.865714e-01,10148.000870,5143.775819,1354.078490,1170.760874,1228.688582,847.390849,542.857025,5143.775819,...,14,4.682655,11.391568,39.460645,22.705991,7.464779,10.957053,3.337309,11775.0,128.058241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67357,3256715.207,3.606835e-01,778636.743500,494467.269400,171473.974900,78482.738560,140150.731800,29683.152010,8075.636418,484972.291400,...,25,4.264945,11.899757,32.215254,17.233975,6.563738,17.911062,9.911268,3200688.0,94.114959
8128,4484868.915,4.643000e-01,8672.694160,5446.238037,2272.327685,1086.102097,1235.970960,551.521255,300.316041,5446.238037,...,14,3.707705,6.737959,22.827046,26.110811,9.528833,19.252558,11.835089,11133.0,154.692979
64715,2139630.638,4.562250e-01,29447.373870,13911.279970,3949.600529,3476.810457,3076.882282,916.572140,2491.414566,13911.279970,...,25,5.540162,13.964955,36.502601,19.193257,6.992789,9.955668,7.850568,72710.0,103.077914
82502,1801332.222,4.405385e-01,11300.391750,6193.793176,1930.219257,1467.223863,1656.835677,0.010391,504.633414,634.870574,...,14,6.961580,11.424701,32.478797,22.983980,5.232501,14.269346,6.649095,13625.0,191.226798


In [ ]:
# plt.figure()
# plt.scatter(df['pc'], df['pop_tmean'])

In [ ]:

columns_to_drop = [ 'population', 'wu_rate'] + extra_drop#'population',
df = dataset.copy()
df = df.dropna(axis = 0)


df
df['pc'] = df['wu_rate']/df['pop']
#df['pc'] = df['pc_median']
#del( df['pc_median'])
if 1:
    df = df[df['pop']>8000]
    mask = (df['pc']>=30) & (df['pc']<=400)
    df = df[mask]
    #df = df[df['LONG']<=-98]
    #df = df[df['pc']<df['p95']]
    #df = df[df['pc']>df['p5']]

#mask = ((df['pc_median']-df['pc'])/df['pc_median']) <0.5
#df = df[mask]

df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('pc')
X =df[features]
#from statsmodels.distributions.empirical_distribution import ECDF
# y = df['pc'].values
# ec = ECDF(y)
# y = ec(y)
y =df['pc']

# 
#plt.figure()
#v_freq = plt.hist(df['pc'].values, bins = 30)
#freq = np.interp(df['pc'].values, v_freq[1][1:], v_freq[0], left=None, right=None, period=None)
#X['freq'] = freq

tar_ec = ce.TargetEncoder(return_df=True, cols = ['county_id'] )
#X = tar_ec.fit_transform(X, y)

#Split data
if 1:
    del(X['sys_id'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 123)
    #Train_freq = X_train['freq']
    #Test_freq = X_test['freq']
    #del(X_test['freq'])
    #del(X_train['freq'])
else:
    if 0:
        east_sys = pd.DataFrame(df[df['LONG']>-98]['sys_id'].unique())
        west_sys = pd.DataFrame(df[df['LONG']<=-98]['sys_id'].unique())
        east_sys_train = east_sys.sample(frac = 0.7, random_state = 123)
        west_sys_train = west_sys.sample(frac = 0.7, random_state = 123)
        trainsys = east_sys_train[0].values.tolist() +  west_sys_train[0].values.tolist()
        east_testsys = east_sys.drop(east_sys_train.index)
        west_testsys = west_sys.drop(west_sys_train.index)
        testsys = east_testsys[0].values.tolist() +  west_testsys[0].values.tolist()
        
        X_train = df[df['sys_id'].isin(trainsys)]
        y_train = X_train['pc']
        del(X_train['pc'])
        del(X_train['sys_id'])

        X_test = df[df['sys_id'].isin(testsys)]
        y_test = X_test['pc']
        del(X_test['pc'])
        del(X_test['sys_id'])
    
    else:
        sys_names = pd.DataFrame(df['sys_id'].unique())
        trainsys = sys_names.sample(frac = 0.7, random_state = 123)
        testsys = sys_names.drop(trainsys.index)

        X_train = df[df['sys_id'].isin(trainsys[0])]
        y_train = X_train['pc']
        del(X_train['pc'])
        del(X_train['sys_id'])

        X_test = df[df['sys_id'].isin(testsys[0])]
        y_test = X_test['pc']
        del(X_test['pc'])
        del(X_test['sys_id'])

    



lgb_params = {
    'n_jobs': 1,
    'max_depth': 8,
    'min_data_in_leaf': 20,
    'subsample': 0.5,
    'n_estimators': 500,
    'learning_rate': 0.2,
    'colsample_bytree': 0.5,
    'boosting_type': 'gbdt',
   
}

# Estimator  -- reg:squaredlogerror, "reg:squarederror"
if 1:
#     gb = xgb.XGBRegressor(objective="reg:squarederror", tree_method = 'hist', colsample_bytree=0.8, learning_rate=0.20,
#                                   max_depth=7, alpha=100,  n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
#                                   seed = 123, reg_lambda=10, min_child_weight = 1, gamma = 10, max_delta_step = 0, 
#                                )
     gb = xgb.XGBRegressor(objective="reg:squarederror", tree_method = 'hist', colsample_bytree=0.8, learning_rate=0.20,
                                   max_depth=7, alpha=100,  n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                                  seed = 123, reg_lambda=10, min_child_weight = 1, gamma = 10, max_delta_step = 0, 
                               )
#      if 0: # log pc
#         gb = xgb.XGBRegressor(objective="reg:squarederror", tree_method = 'hist', colsample_bytree=0.8, learning_rate=0.20,
#                                    max_depth=7, alpha=2,  n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
#                                   seed = 123, reg_lambda=10, gamma = 0,min_child_weight = 3,  max_delta_step = 0, 
#                                )
else:
    gb = LGBMRegressor(objective='regression', alpha=quantile_alpha, **lgb_params)


main_pipeline = []

# (A.2) transformation of categorical features

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# (A.3)do all preprocessing
if 1: # r2 = 0.785
    preprocessor = ColumnTransformer(remainder = 'passthrough',
        transformers=[       
            ("cat", categorical_transformer, categorical_features),
            ('Target encoding', ce.OneHotEncoder( cols = ['county_id',
                                                          'KG_climate_zone']), ['county_id','KG_climate_zone' ])
        ]
    )
    main_pipeline.append(('preprocess', preprocessor))

# drop r2 to 0.774
if 0:
    from feature_engine.encoding import CountFrequencyEncoder
    encoder = CountFrequencyEncoder(encoding_method='frequency',
                             variables=categorical_features)
    main_pipeline.append(('enc', encoder))

# drop r2 to 0.663 for ordered and 0.78 for arbitrary
if 0:    
    from feature_engine.encoding import OrdinalEncoder
    encoder = OrdinalEncoder(encoding_method='ordered',
                             variables=categorical_features)
    main_pipeline.append(('enc', encoder))

#***************************




main_pipeline.append(('model', gb))

model = Pipeline(main_pipeline)



w = np.ones_like(y_train)


kwargs = {model.steps[-1][0] + '__sample_weight': w}
model.fit(X_train, y_train, **kwargs)

plt.figure()
ypredict = model.predict(X_test)

w2 = np.ones_like(y_test)

accuracy = r2_score(y_test, ypredict, sample_weight=w2)
accuracy

plt.scatter(y_test, ypredict, s = 4, c = np.log10(X_test['pop']), cmap = 'jet')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()

plt.figure()
plt.scatter(np.log10(y_test*X_test['pop']), np.log10(ypredict*X_test['pop']), s = 4)
accuracy =  r2_score(np.log10(y_test*X_test['pop']),  np.log10(ypredict*X_test['pop']))
#plt.plot([min(y_test*X_test['pop']), max(ypredict*X_test['pop'])], [min(y_test*X_test['pop']), max(y_test)*X_test['pop']], 'r')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()

plt.colorbar()
plt.show()

In [ ]:
X_train

In [ ]:
plt.figure()
plt.scatter(X_train['LONG'], X_train['LAT'], s = 5)
plt.scatter(X_test['LONG'], X_test['LAT'], s = 5)

In [ ]:
e = np.abs(y_test- ypredict)
mask = e>150

In [ ]:
plt.figure()
plt.scatter(X_test[mask]['LONG'], X_test[mask]['LAT'], c =  X_test[mask]['pop'], cmap = 'jet', s = 4)
plt.colorbar()

In [ ]:
X_test.LONG

In [ ]:
features = X_test.columns
imp = []
for feature in features:
    X_test_permutated = X_test.copy()
    val = X_test_permutated[feature].sample(frac = 1)
    val = val.values
    X_test_permutated[feature] = val
    y_predicted_per = model.predict(X_test_permutated)
    accuracy = r2_score(y_test, y_predicted_per)
    print(accuracy)
    imp.append(accuracy - 0.7714)

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
x = df['pc'].values
ec = ECDF(x)

In [ ]:
ec(x)

In [ ]:
ytrain_predicted = model.predict(X_train)
plt.figure()




accuracy = r2_score(y_train, ytrain_predicted)
accuracy

plt.scatter(y_train, ytrain_predicted, s = 4)
plt.plot([min(y_train), max(y_train)], [min(ytrain_predicted), max(ytrain_predicted)], 'r')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()

plt.colorbar()
plt.show()

## Meaure Importance 

In [ ]:
features = X_test.columns
imp = []
for feature in features:
    X_test_permutated = X_test.copy()
    val = X_test_permutated[feature].sample(frac = 1)
    val = val.values
    X_test_permutated[feature] = val
    y_predicted_per = model.predict(X_test_permutated)
    accuracy = r2_score(y_test, y_predicted_per)
    print(accuracy)
    imp.append(accuracy - 0.7714)
    
imp = np.array(imp)   
plt.figure()
plt.bar(features[np.argsort(imp)][0:50], imp[np.argsort(imp)[0:50]]) 

In [ ]:
ordered_features = features[np.argsort(imp)]

In [ ]:
features = np.array(features)
feat_to_drop = features[np.argsort(imp)][70:]
feat_to_drop = feat_to_drop.tolist()

In [ ]:
feat_to_drop = feat_to_drop.tolist()

In [ ]:
feat_to_drop

In [ ]:
feat_to_drop.remove('KG_climate_zone')

In [ ]:
plt.figure()
plt.bar(features[np.argsort(imp)][0:50], imp[np.argsort(imp)[0:50]])

In [ ]:
imp[np.argsort(imp)[0:50]]

In [ ]:
len(features)-60

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.decomposition import PCA

pipeline = Pipeline([ ('disc', KBinsDiscretizer(n_bins=1000, encode='ordinal', strategy='uniform')),                     
                    ('model', gb)])
kwargs = {pipeline.steps[-1][0] + '__sample_weight': w/w}
pipeline.fit(X_train, y_train, **kwargs)

plt.figure()
ypredict = pipeline.predict(X_test)

w2 = 1.0/Test_freq
#w2 = w2/np.sum(w)
w2 = w2/np.max(w2)

accuracy = r2_score(y_test, ypredict, sample_weight=w2/w2)
accuracy

plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

## Pipeline
### Categorical encoding

In [ ]:
# (A.2) transformation of categorical features
categorical_features = ['HUC2', 'state_id',  'KG_climate_zone', 'county_id' ]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")



In [ ]:
# drop_before_preprocess = ['sys_id']
# categorical_features = ['HUC2', 'state_id',  'KG_climate_zone', 'county_id' ]
# ohc1 = MultiOneHotEncoder(catfeatures = categorical_features )
# dataset = ohc1.transform(dataset)

In [ ]:
# for i in dataset.columns:
#     print(i)

# Estimating Water Use without Log transformation

In [ ]:
# columns to drop
columns_to_drop = ['population', 'sys_id', 'pc']
df = dataset.copy()
df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]

df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('wu_rate')
X =df[features]
y = df['wu_rate']

In [ ]:
df.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

In [ ]:
#squaredlogerror
#squarederror
gb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.8, learning_rate=0.1,
                              max_depth=7, alpha=0.01, n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 123, reg_lambda=0.0)

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
ypredict = gb.predict(X_test)
accuracy = r2_score(y_test, ypredict)

In [ ]:
plt.figure()
accuracy = int(accuracy*100)/100.0
plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual Water Use")
plt.ylabel("Estimated Water Use")
plt.grid()
plt.show()

## Log water use & population 

In [ ]:
# columns to drop
columns_to_drop = ['population', 'sys_id', 'pc']
df = dataset.copy()
df = df[df['wu_rate']>100]
df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]

df = df.drop(columns_to_drop, axis=1)
df['pop'] = np.log10(df['pop'])
df['wu_rate'] = np.log10(df['wu_rate'])
features = list(df.columns)
features.remove('wu_rate')


X =df[features]
y = df['wu_rate']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

In [ ]:
gb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.8, learning_rate=0.01,
                              max_depth=7, alpha=0.01, n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 123, reg_lambda=0.0)

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
ypredict = gb.predict(X_test)
accuracy = r2_score(y_test, ypredict)
accuracy

In [ ]:
plt.figure()
accuracy = int(accuracy*100)/100.0
plt.scatter(10**y_test, 10**ypredict, s = 4)
plt.plot([min(10**y_test), max(10**y_test)], [min(10**y_test), max(10**y_test)], 'r')
plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual Water Use")
plt.ylabel("Estimated Water Use")
plt.grid()
plt.show()

In [ ]:
plt.figure()
pc_test = (10**y_test)/(10**X_test['pop'])
pc_predict =  (10**ypredict)/(10**X_test['pop'])
accuracy = r2_score(pc_test, pc_predict )
plt.scatter(pc_test, pc_predict, s = 4)

plt.plot([min(pc_test), max(pc_test)], [min(pc_test), max(pc_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual Water Use")
plt.ylabel("Estimated Water Use")
plt.grid()
plt.show()

## Quantile Regression

## Use Xgboost

In [ ]:
# # columns to drop
# columns_to_drop = ['population', 'sys_id', 'pc']
# df = dataset.copy()
# df = df[df['wu_rate']>0]
# df['pc'] = df['wu_rate']/df['pop']
# df = df[df['pop']>1000]
# mask = (df['pc']>20) & (df['pc']<400)
# df = df[mask]

# df = df.drop(columns_to_drop, axis=1)
# df['pop'] = np.log10(df['pop'])
# df['wu_rate'] = np.log10(df['wu_rate'])
# features = list(df.columns)
# features.remove('wu_rate')


# X =df[features]
# y = df['wu_rate']

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

In [ ]:
def myfunc(quantile):
    def _myfunc(y_true, y_pred):
        errors = y_pred - y_true
        left_mask = errors < 0
        right_mask = errors > 0

        grad = -quantile * left_mask + (1 - quantile) * right_mask
        hess =  np.ones_like(y_pred)

        return grad, hess
    return _myfunc

In [ ]:
# log cosh quantile is a regularized quantile loss function
def log_cosh_quantile(alpha):
    def _log_cosh_quantile(y_true, y_pred):
        err = y_pred - y_true
        err = np.where(err < 0, alpha * err, (1 - alpha) * err)
        grad = np.tanh(err)
        hess = 1 / np.cosh(err)**2
        hess[hess<0.01] =0.01 
        #hess = np.ones_like(hess)
        
        return grad, hess
    return _log_cosh_quantile

In [ ]:
1 / np.cosh(-1000)**2

In [ ]:
def original_quantile_loss(alpha,delta):
    def _original_quantile_loss(y_true,y_pred):
        x = y_true - y_pred
        grad = (x<(alpha-1.0)*delta)*(1.0-alpha)-((x>=(alpha-1.0)*delta)& (x<alpha*delta) )*x/delta-alpha*(x>alpha*delta)
        hess = ((x>=(alpha-1.0)*delta)& (x<alpha*delta) )/delta 
        return grad,hess
    return _original_quantile_loss

In [ ]:
quantile_alphas = [0.1, 0.5, 0.90]

xgb_quantile_alphas = {}
for quantile_alpha in quantile_alphas:
    # to train a quantile regression, we change the objective parameter and
    # specify the quantile value we're interested in
    gb = xgb.XGBRegressor(objective=log_cosh_quantile(quantile_alpha), colsample_bytree=0.8, learning_rate=0.05,
                              max_depth=5, alpha=0.01, n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 323, reg_lambda=0.01)
    gb.fit(X_train, y_train)
    xgb_quantile_alphas[quantile_alpha] = gb

In [ ]:
plt.figure()
for quantile_alpha, lgb in xgb_quantile_alphas.items():
    ypredict = lgb.predict(X_test)
    plt.scatter(10**y_test, 10**ypredict, s = 4, label = "{}".format
                (quantile_alpha))
plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.legend()
lim = [min(10**y_test), max(10**y_test)]
plt.plot(lim, lim, 'k')
plt.xlabel("Actual Water Use")
plt.ylabel("Estimated Water Use")

## Use Ligh GBM

In [ ]:
lgb_params = {
    'n_jobs': 1,
    'max_depth': 8,
    'min_data_in_leaf': 10,
    'subsample': 0.8,
    'n_estimators': 500,
    'learning_rate': 0.1,
    'colsample_bytree': 0.8,
    'boosting_type': 'gbdt'
}

In [ ]:
quantile_alphas = [0.1, 0.5, 0.90]

lgb_quantile_alphas = {}
for quantile_alpha in quantile_alphas:
    # to train a quantile regression, we change the objective parameter and
    # specify the quantile value we're interested in
    lgb = LGBMRegressor(objective='quantile', alpha=quantile_alpha, **lgb_params)
    lgb.fit(X_train, y_train)
    lgb_quantile_alphas[quantile_alpha] = lgb

In [ ]:
plt.figure()
for quantile_alpha, lgb in lgb_quantile_alphas.items():
    ypredict = lgb.predict(X_test)
    plt.scatter(y_test, ypredict, s = 4, label = "{}".format
                (quantile_alpha))
# plt.gca().set_yscale('log')
# plt.gca().set_xscale('log')
plt.legend()
lim = [min(y_test), max(y_test)]
plt.plot(lim, lim, 'k')
plt.xlabel("Actual Water Use")
plt.ylabel("Estimated Water Use")

# Dimension-reduction

In [ ]:
columns_to_drop = ['population', 'sys_id', 'wu_rate', 'pop', 'households2', 'n_employed', 'n_houses']
df = dataset.copy()

df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]


df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('pc')
X =df[features]
y = df['pc']

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10, whiten = True)
pca.fit(X)

print(pca.explained_variance_ratio_)

print(pca.singular_values_)

In [ ]:
pca.transform(X)

# Testing more ideas for estimating Per Capita 

In [ ]:
def make_dataset(dataset = dataset, filters = [], drop_cols = [] ):
    pass
    


## No action

In [ ]:
 
columns_to_drop = ['population', 'sys_id', 'wu_rate']
df = dataset.copy()

df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]


df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('pc')
X =df[features]
y = df['pc']

#squarederror
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)
gb = xgb.XGBRegressor(objective="reg:squarederror", colsample_bytree=0.8, learning_rate=0.1,
                              max_depth=7, alpha=0.01, n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 123, reg_lambda=0.0)

w = np.exp(1e-2*np.power((y_train-200)/10, 2.0))
w = w/np.sum(w)
w = w.values
w = w/np.max(w)
gb.fit(X_train, y_train, sample_weight=w/w)
plt.figure()
ypredict = gb.predict(X_test)

w2 = np.exp(1e-3*np.power((y_test-200)/10, 2.0))
w2 = w2/np.sum(w)
w2 = w2.values
w2 = w2/np.max(w2)

accuracy = r2_score(y_test, ypredict, sample_weight=w2/w2)
accuracy

plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

In [ ]:

columns_to_drop = ['population', 'sys_id', 'wu_rate']
df = dataset.copy()
del(df['pc_median'])
df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>1000]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]


df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('pc')
X =df[features]
y = df['pc']

# 
plt.figure()
v_freq = plt.hist(df['pc'].values, bins = 30)
freq = np.interp(df['pc'].values, v_freq[1][1:], v_freq[0], left=None, right=None, period=None)
X['freq'] = freq
#squarederror
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 123)
Train_freq = X_train['freq']
Test_freq = X_test['freq']
del(X_test['freq'])
del(X_train['freq'])

#squarederror
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)
gb = xgb.XGBRegressor(objective="reg:squarederror", tree_method = 'hist', colsample_bytree=0.8, learning_rate=0.20,
                              max_depth=7, alpha=100,  n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 123, reg_lambda=10, min_child_weight = 1, gamma = 10, max_delta_step = 0, 
                           )
#gb = xgb.XGBRegressor(objective="reg:squarederror" )


w = 1.0/Train_freq
#w = w/np.sum(w)
w = w/np.max(w)
gb.fit(X_train, y_train, sample_weight=w/w)
plt.figure()
ypredict = gb.predict(X_test)

w2 = 1.0/Test_freq
#w2 = w2/np.sum(w)
w2 = w2/np.max(w2)

accuracy = r2_score(y_test, ypredict, sample_weight=w2/w2)
accuracy

plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

# Use Pipeline 


In [ ]:


# (A.1) transformation of numeric features
numeric_features = ["age", "fare"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

# (A.2) transformation of categorical features
categorical_features = ["embarked", "sex", "pclass"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")


# (A.3)do all preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

#  (B.1) complete pipeline
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

In [ ]:
from sklearn import set_config

set_config(display="diagram")
clf

In [ ]:
## Scaling

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

pipeline = Pipeline([ ('robost_scaler', RobustScaler()),                     
                    ('model', gb)])
kwargs = {pipeline.steps[-1][0] + '__sample_weight': w/w}
pipeline.fit(X_train, y_train, **kwargs)

plt.figure()
ypredict = pipeline.predict(X_test)

w2 = 1.0/Test_freq
#w2 = w2/np.sum(w)
w2 = w2/np.max(w2)

accuracy = r2_score(y_test, ypredict, sample_weight=w2/w2)
accuracy

plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

from sklearn.decomposition import PCA

pipeline = Pipeline([ ('uniform_dist', PowerTransformer()),                     
                    ('model', gb)])
kwargs = {pipeline.steps[-1][0] + '__sample_weight': w/w}
pipeline.fit(X_train, y_train, **kwargs)

plt.figure()
ypredict = pipeline.predict(X_test)

w2 = 1.0/Test_freq
#w2 = w2/np.sum(w)
w2 = w2/np.max(w2)

accuracy = r2_score(y_test, ypredict, sample_weight=w2/w2)
accuracy

plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.decomposition import PCA

pipeline = Pipeline([ ('disc', KBinsDiscretizer(n_bins=1000, encode='ordinal', strategy='uniform')),                     
                    ('model', gb)])
kwargs = {pipeline.steps[-1][0] + '__sample_weight': w/w}
pipeline.fit(X_train, y_train, **kwargs)

plt.figure()
ypredict = pipeline.predict(X_test)

w2 = 1.0/Test_freq
#w2 = w2/np.sum(w)
w2 = w2/np.max(w2)

accuracy = r2_score(y_test, ypredict, sample_weight=w2/w2)
accuracy

plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

In [ ]:
# plot feature importance
# ‘weight’: the number of times a feature is used to split the data across all trees.

# ‘gain’: the average gain across all splits the feature is used in.

# ‘cover’: the average coverage across all splits the feature is used in.

# ‘total_gain’: the total gain across all splits the feature is used in.

# ‘total_cover’: the total coverage across all splits the feature is used in.
plot_importance(gb,   xlabel='total_gain', importance_type='total_gain', max_num_features = 10)
from sklearn.feature_selection import SelectFromModel
plt.tight_layout()

In [ ]:
# Another look at importance

In [ ]:
plt.figure()
ypredict = gb.predict(X_test)

w2 = 1.0/Test_freq
#w2 = w2/np.sum(w)
w2 = w2/np.max(w2)

accuracy = r2_score(y_test, ypredict, sample_weight=w2/w2)
accuracy

plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

In [ ]:
# Tuning
gb.get_params()
# params = {'colsample_bylevel': scipy.stats.uniform (loc = 0.1, scale=0.9),
#     'colsample_bynode': scipy.stats.uniform (loc = 0.1, scale=0.9),
#     'colsample_bytree': scipy.stats.uniform (loc = 0.1, scale=0.9),
#     'gamma': scipy.stats.uniform (loc = 0, scale=100),
#     'learning_rate': scipy.stats.uniform (loc = 0.001, scale=0.4),
#     'max_depth': [3,4,5,6,7,8,9, 10],
#     'min_child_weight' : scipy.stats.uniform (loc = 0 scale=1000),
#     'reg_alpha': 100,
#      'reg_lambda': 10,
#      'subsample': 0.8
#     }

In [ ]:
from pycaret.regression import *

columns_to_drop = ['population', 'sys_id', 'wu_rate']
df = dataset.copy()
del(df['pc_median'])
df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>1000]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]


df = df.drop(columns_to_drop, axis=1)
data = df.sample(frac=0.9, random_state=786)
data_unseen = df.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

exp0 = setup(data = data2, target = 'pc', train_size = 0.8,                   
                   fold_shuffle= True, data_split_shuffle=True, session_id=123)
xgb = create_model('xgboost', fold = 3)
tuned_ada = tune_model(xgb)

In [ ]:
tuned_ada = tune_model(xgb)

In [ ]:
columns_to_drop = ['population', 'sys_id', 'wu_rate']
df = dataset.copy()

df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]


df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('pc')
X =df[features]
y = df['pc']

In [ ]:
columns_to_drop = ['population', 'sys_id', 'wu_rate']
df = dataset.copy()

df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]


df = df.drop(columns_to_drop, axis=1)
features = list(df.columns)
features.remove('pc')
X =df[features]
y = df['pc']

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20, whiten = True)
pca.fit(X)

print(pca.explained_variance_ratio_)

print(pca.singular_values_)
X = pca.transform(X)

In [ ]:
#squarederror
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)
gb = xgb.XGBRegressor(objective='reg:"squarederror"', colsample_bytree=0.8, learning_rate=0.1,
                              max_depth=7, alpha=0.01, n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 123, reg_lambda=0.0)
gb.fit(X_train, y_train)

In [ ]:
plt.figure()
ypredict = gb.predict(X_test)
accuracy = r2_score(y_test, ypredict)
accuracy
accuracy = int(accuracy*100)/100.0
plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

## Quantile transformation of the target PC
### Uniform Distribution

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import quantile_transform
from scipy import stats
x = pd.DataFrame(np.random.rand(100), columns = ['x'])
stats.percentileofscore(x['x'], 0.47219 )

In [ ]:
from scipy import stats

In [ ]:
# quantile transform
# columns to drop
columns_to_drop = ['population', 'sys_id', 'wu_rate']
df = dataset.copy()

df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]
pc_max = df['pc'].max()
pc_min = df['pc'].min()

df = df.drop(columns_to_drop, axis=1)
wu = quantile_transform(df['pc'].values.reshape(len(df),1), n_quantiles=5000, 
                        random_state=0) #, output_distribution = 'normal'
df['pc'] = wu.flatten()
features = list(df.columns)
features.remove('pc')
X =df[features]
y = df['pc']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

In [ ]:
#squarederror
gb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.8, learning_rate=0.1,
                              max_depth=7, alpha=0.01, n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 123, reg_lambda=0.0)

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
plt.figure()
ypredict = gb.predict(X_test)
accuracy = r2_score(y_test, ypredict)
accuracy
accuracy = int(accuracy*100)/100.0
plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual Water Use")
plt.ylabel("Estimated Water Use")
plt.grid()
plt.show()

In [ ]:
plt.figure()
ypredict = gb.predict(X_test)
ypredict = pc_min + ypredict*(pc_max-pc_min)
y_test = pc_min + y_test*(pc_max-pc_min)
accuracy = r2_score(y_test, ypredict)
accuracy
accuracy = int(accuracy*100)/100.0
plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

## Normal Distribution

In [ ]:
# quantile transform
# columns to drop
columns_to_drop = ['population', 'sys_id', 'wu_rate']
df = dataset.copy()

df['pc'] = df['wu_rate']/df['pop']
df = df[df['pop']>100]
mask = (df['pc']>20) & (df['pc']<500)
df = df[mask]
pc_mean = df['pc'].mean()
pc_std= df['pc'].std()

df = df.drop(columns_to_drop, axis=1)
wu = quantile_transform(df['pc'].values.reshape(len(df),1), n_quantiles=5000, 
                        random_state=0, output_distribution = 'normal') #, output_distribution = 'normal'
df['pc'] = wu.flatten()
features = list(df.columns)
features.remove('pc')
X =df[features]
y = df['pc']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

In [ ]:
gb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.8, learning_rate=0.1,
                              max_depth=7, alpha=0.01, n_estimators=500, rate_drop=0.9, skip_drop=0.5, subsample=0.8,
                              seed = 123, reg_lambda=0.0)

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
plt.figure()
ypredict = gb.predict(X_test)
accuracy = r2_score(y_test, ypredict)
accuracy
accuracy = int(accuracy*100)/100.0
plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
#plt.gca().set_yscale('log')
#plt.gca().set_xscale('log')
plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()

In [ ]:
pc_std

In [ ]:
plt.figure()
ypredict = gb.predict(X_test)

ypredict= 180+pc_mean + pc_std * ypredict
y_test = 180+pc_mean + pc_std * y_test
accuracy = r2_score(y_test, ypredict)

accuracy = int(accuracy*100)/100.0
plt.scatter(y_test, ypredict, s = 4)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')

plt.title("$R^2 = ${}".format(accuracy))
plt.xlabel("Actual PC Water Use")
plt.ylabel("Estimated PC Water Use")
plt.grid()
plt.show()